In [3]:
import numpy as np
from sklearn.preprocessing import normalize

In [29]:
data_path = './wine.csv'
data = np.genfromtxt(data_path, delimiter=',', skip_header =1)

print(data)

[[1.000e+00 1.423e+01 1.710e+00 ... 1.040e+00 3.920e+00 1.065e+03]
 [1.000e+00 1.320e+01 1.780e+00 ... 1.050e+00 3.400e+00 1.050e+03]
 [1.000e+00 1.316e+01 2.360e+00 ... 1.030e+00 3.170e+00 1.185e+03]
 ...
 [3.000e+00 1.327e+01 4.280e+00 ... 5.900e-01 1.560e+00 8.350e+02]
 [3.000e+00 1.317e+01 2.590e+00 ... 6.000e-01 1.620e+00 8.400e+02]
 [3.000e+00 1.413e+01 4.100e+00 ... 6.100e-01 1.600e+00 5.600e+02]]


In [5]:
np.delete(data,0,1)


array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [6]:
sum_of_rows = data.sum(axis=1)
normalized_data = data / sum_of_rows[:, np.newaxis]

print(normalized_data)

[[8.02568218e-04 1.14205457e-02 1.37239165e-03 ... 8.34670947e-04
  3.14606742e-03 8.54735152e-01]
 [8.36750063e-04 1.10451008e-02 1.48941511e-03 ... 8.78587566e-04
  2.84495021e-03 8.78587566e-01]
 [7.44701449e-04 9.80027107e-03 1.75749542e-03 ... 7.67042493e-04
  2.36070359e-03 8.82471217e-01]
 ...
 [2.95793812e-03 1.30839463e-02 4.21999172e-03 ... 5.81727830e-04
  1.53812782e-03 8.23292777e-01]
 [2.94993953e-03 1.29502345e-02 2.54678112e-03 ... 5.89987905e-04
  1.59296734e-03 8.25983067e-01]
 [4.16319734e-03 1.96086595e-02 5.68970303e-03 ... 8.46516792e-04
  2.22037191e-03 7.77130169e-01]]


In [25]:
# 0 - unchecked
# -1 - noise 
# 1,2... - number of cluster

def _DBSCAN(dataset, eps, min_points):
    labels = np.zeros(len(dataset), dtype=int)
    current_cluster = 0
    
    for p in range(0, len(dataset)):
        if labels[p] != 0:
           continue
        
        neighbor_points = findNeighbors(dataset, p, eps)
        if len(neighbor_points) < min_points:
            labels[p] = -1   
        else: 
           current_cluster += 1
           expandCluster(dataset, labels, p, neighbor_points, current_cluster, eps, min_points)
            
    return labels


def expandCluster(dataset, labels, p, neighbor_points, current_cluster, eps, min_points):
    labels[p] = current_cluster
    
    i = 0
    while i < len(neighbor_points):      
        Pn = neighbor_points[i]
        if labels[Pn] == -1:
           labels[Pn] = current_cluster
        
        elif labels[Pn] == 0:
            labels[Pn] = current_cluster
            neighbor_points_next = findNeighbors(dataset, Pn, eps)
            
            if len(neighbor_points_next) >= min_points:
                neighbor_points = neighbor_points + neighbor_points_next
        i += 1        

def findNeighbors(dataset, p, eps):
    neighbors = []
    for Pn in range(0, len(dataset)):
        if np.linalg.norm(dataset[p] - dataset[Pn]) < eps:
           neighbors.append(Pn)
            
    return neighbors


In [26]:
_labels = _DBSCAN(normalized_data, eps=0.014, min_points=18)
print(_labels)

[-1  1  1  1  2  1  1  1  1  1  1  1 -1  1 -1  1  1  1 -1  2  2  2  1  1
 -1  2  1  1 -1  1  1  1  1  1  1 -1 -1  1  1  2  2  1  1  2 -1  1  1  1
  1  1  1  1  1  1  1  1 -1  1  1  2  2 -1  2 -1 -1  2  2  2  2 -1 -1 -1
 -1  2 -1 -1 -1 -1 -1 -1 -1  2  2  2  3 -1  3  2  2 -1  2 -1  2 -1 -1 -1
 -1 -1  2 -1  2  2 -1 -1  2 -1  2  2 -1  2  3 -1  2 -1 -1 -1  2 -1 -1  2
  2 -1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1  2  2  2  2  2 -1  2  2  2 -1  3  2
  2  2 -1  2  2 -1 -1 -1 -1  2  2  2 -1 -1  2  2  2  2  3  2  2  2  2  2
  2  3  3 -1  2  2  2  2  2  3]


In [27]:
from sklearn.cluster import DBSCAN
labels = DBSCAN(eps=0.014, min_samples=18).fit(normalized_data).labels_
print(labels)

[-1  0  0  0  1  0  0  0  0  0  0  0 -1  0 -1  0  0  0 -1  1  1  1  0  0
 -1  1  0  0 -1  0  0  0  0  0  0 -1 -1  0  0  1  1  0  0  1 -1  0  0  0
  0  0  0  0  0  0  0  0 -1  0  0  1  1 -1  1 -1 -1  1  1  1  1 -1 -1 -1
 -1  1 -1 -1 -1 -1 -1 -1 -1  1  1  1  2 -1  2  1  1 -1  1 -1  1 -1 -1 -1
 -1 -1  1 -1  1  1 -1 -1  1 -1  1  1 -1  1  2 -1  1 -1 -1 -1  1 -1 -1  1
  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  1  1  1  1  1 -1  1  1  1 -1  2  1
  1  1 -1  1  1 -1 -1 -1 -1  1  1  1 -1 -1  1  1  1  1  2  1  1  1  1  1
  1  2  2 -1  1  1  1  1  1  2]
